In [1]:
%load_ext autoreload
%autoreload 2


In [19]:
import sys
import os
sys.path.append('..')

import torch
import optuna
import pytorch_lightning as pl

from radianet.datasets import SampleDataset3D
from radianet.config import Config
from radianet.models import SimpleCNN
from radianet.callbacks import MetricsCallback
from radianet.lightningnet import LightningNet

In [ ]:
config = Config

def objective(trial):
    
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        os.path.join(config.MODEL_DIR, "trial_{}".format(trial.number), "{epoch}"), 
        monitor="val_loss", save_top_k = -1
    )
    metrics_callback = MetricsCallback()

    trainer = pl.Trainer(
        logger=False,
        max_epochs=config.EPOCHS,
        checkpoint_callback=checkpoint_callback,
        callbacks=[metrics_callback],
        gpus=0 if torch.cuda.is_available() else None
    )

    n_sample = 5
    datasets = SampleDataset3D(n_sample = n_sample)
    datasets = datasets.get_dataset()

    lightning_model = LightningNet(trial, config, SimpleCNN, datasets)
    trainer.fit(lightning_model)

    return metrics_callback.metrics[-1]["val_loss"]

study = optuna.create_study()
study.optimize(objective, n_trials= config.N_TRIALS)


INFO:lightning:GPU available: True, used: False
INFO:lightning:
   | Name                       | Type        | Params
-------------------------------------------------------
0  | model                      | SimpleCNN   | 1 M   
1  | model.conv_block           | Sequential  | 300   
2  | model.conv_block.0         | Conv3d      | 280   
3  | model.conv_block.1         | BatchNorm3d | 20    
4  | model.conv_block.2         | ReLU        | 0     
5  | model.conv_block.3         | MaxPool3d   | 0     
6  | model.conv_block.4         | Dropout     | 0     
7  | model.feedforward_block    | Sequential  | 1 M   
8  | model.feedforward_block.0  | Linear      | 1 M   
9  | model.feedforward_block.1  | ReLU        | 0     
10 | model.feedforward_block.2  | Dropout     | 0     
11 | model.feedforward_block.3  | Linear      | 2 K   
12 | model.feedforward_block.4  | ReLU        | 0     
13 | model.feedforward_block.5  | Dropout     | 0     
14 | model.feedforward_block.6  | Linear      | 1 K   


In [15]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 5
Best trial:
  Value: 0.4275152087211609
  Params: 
    n_fw_layers: 2
    output_fw_dim0: 108.1097334448185
    output_fw_dim1: 5.654177889007046
    dropout: 0.2810253474408885
    lr: 0.00012853161511888015
